In [ ]:
from Classifications import Classifications
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import InputLayer, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Sequential
from scipy.spatial.transform import Rotation as R
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import compute_class_weight
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
classifications = Classifications()
model_h_path = "../ModelData.h"
data_csv = "sensor_data_cleaned.csv"
batch_size = 128
num_epochs = 128
patience = 16
seed = 42
rng = np.random.default_rng(seed=seed)
test_split = 0.2
val_split = 0.2

In [ ]:
model = Sequential([
    InputLayer((classifications.num_steps, classifications.num_features)),
    Conv1D(16, 3, activation="relu"),
    MaxPooling1D(2),
    Conv1D(32, 3, activation="relu"),
    GlobalAveragePooling1D(),
    Dense(classifications.num_classes, activation=None),
])

model.summary()

In [ ]:
sample_input = tf.random.uniform(shape=(1, classifications.num_steps, classifications.num_features), dtype=tf.float16, seed=seed)
logits = model.predict(sample_input)

In [ ]:
prediction = tf.argmax(logits, axis=1)[0]
print(prediction)

In [ ]:
prediction_string = ' '.join(reversed(classifications.classes[prediction]))
print(f"Prediction: {prediction_string}")

In [ ]:
df = pd.read_csv(data_csv)
df = pd.concat([df, df], ignore_index=True)

# Peak detection
peaks, _ = find_peaks(
    np.sum(df[["ax", "ay", "az"]].values ** 2, axis=1),
    height=classifications.squared_acceleration_threshold,
    distance=classifications.num_steps,
)

sensor_columns = ["ax", "ay", "az", "gx", "gy", "gz"]

X = []
y = []

for peak in peaks:
    start_idx = peak - classifications.steps_before_peak
    end_idx = peak + classifications.steps_after_peak
    if start_idx < 0 or end_idx >= len(df):
        continue

    shot_df = df.loc[start_idx:end_idx]
    assert len(shot_df) == classifications.num_steps

    shot_data = shot_df[sensor_columns].values
    stroke = df.loc[peak, "stroke"].lower()
    side = df.loc[peak, "side"].lower()
    spin = df.loc[peak, "spin"].lower()
    label_key = (stroke, side, spin)

    label = classifications.class_to_idx[label_key]
    X.append(shot_data)
    y.append(label)

X = np.array(X).astype(np.float16)
y = np.array(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_split, random_state=seed, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_split, random_state=seed, stratify=y_temp)

scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, X_train.shape[-1]))

def transform_data(X, scaler):
    num_samples, num_steps, num_features = X.shape
    X_flat = X.reshape(-1, num_features)
    X_scaled_flat = scaler.transform(X_flat)
    return X_scaled_flat.reshape(num_samples, num_steps, num_features)

X_train = transform_data(X_train, scaler)
X_val = transform_data(X_val, scaler)
X_test = transform_data(X_test, scaler)

def random_rotate_sample(sample, label):
    def rotate_fn(sample_np):
        random_rotation = R.random(rng=rng)
        rotated_accel = random_rotation.apply(sample_np[:, :3])
        rotated_gyro = random_rotation.apply(sample_np[:, 3:])
        rotated_sample = np.concatenate((rotated_accel, rotated_gyro), axis=1)
        return rotated_sample.astype(np.float16)

    rotated_sample = tf.py_function(func=rotate_fn, inp=[sample], Tout=tf.float16)
    rotated_sample.set_shape(sample.shape)
    return rotated_sample, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(len(X_train), seed=seed)
    .map(random_rotate_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
#import matplotlib.pyplot as plt
#
#sample, label = next(iter(train_ds))
#sample = sample[0]
#label = label[0]
#
#plot_width = 6
#x_ticks = np.arange(len(sample))
#
## Raw data plot
#plots = (
#    { "label": "ax", "title": "Acceleration X", "color": "r" },
#    { "label": "ay", "title": "Acceleration Y", "color": "g" },
#    { "label": "az", "title": "Acceleration Z", "color": "b" },
#    { "label": "gx", "title": "Gyroscope X", "color": "r" },
#    { "label": "gy", "title": "Gyroscope Y", "color": "g" },
#    { "label": "gz", "title": "Gyroscope Z", "color": "b" },
#)
#
#fig, axes = plt.subplots(len(plots), 1, figsize=(plot_width, 3 * len(plots)))
#for i, (ax, plot) in enumerate(zip(axes, plots)):
#    ax.plot(x_ticks, sample[:, i], label=plot["title"], color=plot["color"])
#    ax.set_xlabel("Milliseconds")
#    ax.set_ylabel(plot["title"])
#    ax.set_xticks(x_ticks[::3])
#    ax.ticklabel_format(style='plain')
#    ax.grid(True)
#
#plt.tight_layout()
#plt.show()

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint_callback = ModelCheckpoint(
    filepath="best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
)

early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=patience,
    restore_best_weights=True,
    verbose=1,
)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[np.where(np.unique(y_train) == i)[0][0]] if i in np.unique(y_train) else 0 for i in range(classifications.num_classes)}

history = model.fit(
    train_ds,
    epochs=num_epochs,
    callbacks=[checkpoint_callback, early_stopping],
    validation_data=val_ds,
    class_weight=class_weights,
    verbose=1,
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
print(f"Model size: {len(tflite_model)} bytes")

with open(model_h_path, "w") as f:
    f.write("#ifndef _MODELDATA_H_\n#define _MODELDATA_H_\n")
    f.write("const unsigned char model[] = {")
    f.write(",".join(f"0x{b:02x}" for b in tflite_model))
    f.write("};\n#endif\n")